#### Exercise 1

In [41]:
import re
import pandas as pd
import requests
#from io import StringIOrequests

In [42]:
requests.__version__

'2.27.1'

### A

In [43]:
cpr_pattern = re.compile(r'(\d{2})(\d{2})(\d{2})[-]?(\d{4})')

# Example usage:
cpr_number_1 = '0102031234'
cpr_number_2 = '010203-1234'

In [44]:
test_1 = cpr_pattern.match(cpr_number_1)
test_2 = cpr_pattern.match(cpr_number_2)

if test_1:
    print("Test 1!")
    print("Test 1 groups:", test_1.groups())
    day, month, year, identifier = test_1.groups()
    print(f"DD: {day}, MM: {month}, YY: {year}, IIII: {identifier}")

if test_2:
    print("Test 2!")
    day, month, year, identifier = test_2.groups()
    print(f"DD: {day}, MM: {month}, YY: {year}, IIII: {identifier}")

Test 1!
Test 1 groups: ('01', '02', '03', '1234')
DD: 01, MM: 02, YY: 03, IIII: 1234
Test 2!
DD: 01, MM: 02, YY: 03, IIII: 1234


### B

In [45]:
def cpr_century(cpr):
    cpr_pattern = re.compile(r'(\d{2})(\d{2})(\d{2})[-]?(\d{4})')
    match = cpr_pattern.match(cpr)

    if match:
        day, month, year, identifier = match.groups()
        identifier = int(identifier)
        year = int(year)

        if 1 <= identifier <= 3999:
            century = 1900 + year // 100
        elif 4000 <= identifier <= 4999 and 0 <= year <= 36:
            century = 2000 + year // 100
        elif 4000 <= identifier <= 4999 and 37 <= year <= 99:
            century = 1900 + year // 100
        elif 5000 <= identifier <= 8999 and 0 <= year <= 57:
            century = 2000 + year // 100
        elif 5000 <= identifier <= 8999 and 58 <= year <= 99:
            century = 1800 + year // 100
        elif 9000 <= identifier <= 9999 and 0 <= year <= 36:
            century = 2000 + year // 100
        elif 9000 <= identifier <= 9999 and 37 <= year <= 99:
            century = 1900 + year // 100
        else:
            raise ValueError("Invalid identifier or year range")
        
        return century
    else:
        raise ValueError("Invalid CPR format")

cpr_number = '2201978999'
try:
    result = cpr_century(cpr_number)
    print(f"The person was born in the {result}s.")
except ValueError as e:
    print(f"Error: {e}")


The person was born in the 1800s.


In [46]:
cpr_lst = ['220197-8989', '2201650099', '121201-0976', '1224230973', '010124-9001']

In [47]:
for cpr in cpr_lst:
    result = cpr_century(cpr)
    print(fr'CPR:{cpr} is born in the {result}')

CPR:220197-8989 is born in the 1800
CPR:2201650099 is born in the 1900
CPR:121201-0976 is born in the 1900
CPR:1224230973 is born in the 1900
CPR:010124-9001 is born in the 2000


In [48]:
from io import StringIO


In [49]:
# URL of the CSV file
url = 'https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv'

# Fetching the content from the URL
response = requests.get(url)

# Checking if the request was successful (status code 200)
if response.status_code == 200:
    # Reading CSV data using pandas
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)

    # Displaying the first few rows of the DataFrame
    #print(df.head())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


In [50]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        250 non-null    int64  
 1   id                250 non-null    int64  
 2   domain            250 non-null    object 
 3   type              238 non-null    object 
 4   url               250 non-null    object 
 5   content           250 non-null    object 
 6   scraped_at        250 non-null    object 
 7   inserted_at       250 non-null    object 
 8   updated_at        250 non-null    object 
 9   title             250 non-null    object 
 10  authors           170 non-null    object 
 11  keywords          0 non-null      float64
 12  meta_keywords     250 non-null    object 
 13  meta_description  54 non-null     object 
 14  tags              27 non-null     object 
 15  summary           0 non-null      float64
dtypes: float64(2), int64(2), object(12)
memory u

There a multiple all null columns. #11 and #15


Missing Values:

The 'authors', 'keywords', 'meta_keywords', 'meta_description', and 'tags' columns contain empty lists ('[]'), which may imply missing or incomplete information. You may need to handle or investigate these cases.
Date Representation:

The 'scraped_at', 'inserted_at', and 'updated_at' columns seem to contain timestamp values. Ensure that these columns are correctly parsed as datetime objects if you plan to perform temporal analysis.
Text Data Cleaning:

The 'content' column contains HTML tags, line breaks, and special characters. Depending on your analysis, you might want to clean or preprocess the text data to remove HTML tags and unnecessary characters.
Categorical Values:

The 'type' column appears to categorize news into 'unreliable' and 'fake.' Ensure that these categories are well-defined and consistent throughout the dataset.
Column Names:

Some column names contain spaces, which might cause inconvenience in handling them. You might consider renaming columns for ease of use.
Encoding Issues:

Check for any encoding issues in the text data, especially if the content includes non-ASCII characters. Ensure that the text is encoded properly.
Duplicate Titles:

Ensure that the 'title' column does not contain duplicate entries, as each news article should have a unique title.
Column Data Types:

Confirm that each column has the correct data type. For example, 'type' might be categorical, 'scraped_at,' 'inserted_at,' and 'updated_at' should be datetime, etc.

In [51]:
# Explore unique values in categorical columns
# for column in df.select_dtypes(include='object').columns:
    # print(f"Unique values in {column}: {df[column].unique()}")


In [52]:
def clean_text(raw_text):
    cleaned_text = raw_text.lower()
    return cleaned_text

In [59]:
def clean_text(raw_text):
   cleaned_text = re.sub(r'\s+', ' ', raw_text)

    # Replace uppercase letters with lowercase
   cleaned_text = re.sub(r'[A-Z]', lambda match: match.group().lower(), cleaned_text)
   
   cleaned_text = re.sub(r'\b\d+\b', '<NUM>', cleaned_text)
   cleaned_text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '<EMAIL>', cleaned_text)
   cleaned_text = re.sub(r'https?://\S+', '<URL>', cleaned_text)
   cleaned_text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '<DATE>', cleaned_text)


   return cleaned_text


In [60]:
df['content'] = df['content'].apply(clean_text)


In [61]:
df['content']

0      sometimes the power of christmas will make you...
1      awakening of <NUM> strands of dna – “reconnect...
2      never hike alone: a friday the 13th fan film u...
3      when a rare shark was caught, scientists were ...
4      donald trump has the unnerving ability to abil...
                             ...                        
245    prison for rahm, god’s work and many others he...
246    <NUM> useful items for your tiny home headline...
247    former cia director michael hayden said thursd...
248    antonio sabato jr. says hollywood's liberal el...
249    former u.s. president bill clinton on monday c...
Name: content, Length: 250, dtype: object